# Optimizacion de Desarolladores por Seniority



## Variables Exogenas

### Datos

* **IA**: Intervalo entre Arribos [minutos]
* **TA**: Tiempo de Atencion [minutos]

Estan dados en base a distintas funciones de distribucion de probabilidad, obtenidas mediante _fitting_ de datos reales.


In [ ]:
import dataclasses

from scipy.stats import stats


def time_between_arrivals() -> int:
    """
    Provides the next arrival the time delta for the next arrival
    
    Returns:
        int: how many minutes until the next arrival
    """
    best_params = {'b': 8.081771083639762, 'loc': -400.05903534867156, 'scale': 400.0590353486715}
    [next_arrival_in] = stats.pareto.rvs(**best_params, size=1)
    return next_arrival_in


def service_time() -> int:
    """
    Provides the time delta for the next service
    
    Returns:
        int: how many minutes the service will take
    """
    best_params = {'a': 1.0849480846709834, 'b': 1.3832003644016413, 'c': 1.9794855101439612, 'z': 0.982926014368478,
                   'loc': 0.9477414716993133, 'scale': 1296.2750378147014}
    [busy_time] = stats.pareto.rvs(**best_params, size=1)
    return busy_time

### Control

* **NPS**: Numero de Programadores Sr
* **NPJ**: Numero de Programadores Jr

In [ ]:
@dataclasses.dataclass
class ControlVector:
    """
    Control vector for the simulation
    
    Attributes:
        nps (int): Number of Senior Programmers
        npj (int): Number of Junior Programmers
    """
    nps: int
    npj: int

## Variables Endogenas

### Estado

* **NSH**: Numero de tickets en cola de prioridad alta
* **NSL**: Numero de tickets en cola de prioridad baja

In [ ]:
@dataclasses.dataclass
class StatusVector:
    """
    Status vector for the simulation
    
    Attributes:
        nsh (int): Number of high priority tickets
        nsl (int): Number of low priority tickets
    """
    nsh: int
    nsl: int

### Resultados

* **PPS**: Promedio de Permanencia en el Sistema [minutos]
* **PEC**: Promedio de Permanencia en la Cola [minutos]
* **PTO**: Promedio de Tiempo Ocioso [minutos]
* **PTTS**: Porcentaje de Tickets de Baja Prioridad Atendidos por Sr [%]

In [ ]:
@dataclasses.dataclass
class ResultVector:
    """
    Result vector for the simulation
    
    Attributes:
        pps (float): Average time in the system
        pec (float): Average time in the queue
        pto (float): Average idle time
        ptts (float): Percentage of low priority tickets attended by Sr
    """
    pps: float
    pec: float
    pto: float
    ptts: float

## Tabla de Eventos Indepedientes

| Evento       | Evento Futuro No Condicionado | Evento Futuro Condicionado | Condicion                                 |
|--------------|-------------------------------|----------------------------|-------------------------------------------|
| LLEGADA      | LLEGADA                       | Salida Sr[i]               | NSH <= NPS \|\| (NSL > NPJ && NSH <= NPS) |
| ...          | ...                           | Salida Jr[j]               | NSL <= NPJ                                |
| Salida Sr[i] |                               | Salida Sr[i]               | NSH > NPS \|\| (NSL > NPJ && NSH > NPS)   |
| Salida Jr[j] |                               | Salida Jr[j]               | NSL > NPJ                                 |

## Tabla de Eventos Futuros

TPLL (Tiempo de Llegada del Proximo Ticket), TPSSi (Tiempo de Proxima Salida del Programador Sr i-esimo), TPSJj (Tiempo de Proximo Salida del Programador j-esimo)

In [ ]:
@dataclasses.dataclass
class FutureEventsVector:
    """
    Future events vector for the simulation
    
    Attributes:
        tpll (int): Time of the next ticket arrival
        tpss (int): Time of the next Senior Programmer exit
        tpsj (int): Time of the next Junior Programmer exit
    """
    tpll: int
    tpss: list[int]
    tpsj: list[int]